In [5]:
!pip install qiskit

You should consider upgrading via the 'c:\users\dell\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [8]:
import random
import numpy as np

from qiskit import QuantumCircuit, execute, Aer, IBMQ, BasicAer
from qiskit.visualization import plot_histogram, plot_state_qsphere as plot_q
from qiskit.quantum_info import Statevector



from itertools import product #this one I use to find the prob of 00 without sampling. 



In [9]:
def rand_numb(low_in,high_in):
    return np.random.uniform(low=low_in, high=np.nextafter(high_in, np.inf))

#just to keep the upper limit included in the random number generation. 

In [10]:
unitary_param_1 = [0.21249364, 2.33465111, 1.93219486, 2.55074749, 3.67705258, 6.15636742,
 2.26177908, 5.76752756, 2.01463033, 3.33610176]
unitary_param_2 = [3.80940494, 5.70236131, 0.42946852, 3.85949314, 4.95013009, 4.76411749,
 4.31178296, 3.81034009, 1.91721159, 4.63532783]

In [11]:
# First unitary
size = 5
qc1 = QuantumCircuit(size)
for i in range(size):
  qc1.rx(unitary_param_1[i], i)

for i in range(size):
  qc1.ry(unitary_param_1[i+size], i)

for i in range(size-1):
  qc1.cx(i, i+1)

qc1.draw()

┌─────────────┐┌────────────┐                    
q_0: ┤ RX(0.21249) ├┤ RY(6.1564) ├──■─────────────────
     └┬────────────┤├────────────┤┌─┴─┐               
q_1: ─┤ RX(2.3347) ├┤ RY(2.2618) ├┤ X ├──■────────────
      ├────────────┤├────────────┤└───┘┌─┴─┐          
q_2: ─┤ RX(1.9322) ├┤ RY(5.7675) ├─────┤ X ├──■───────
      ├────────────┤├────────────┤     └───┘┌─┴─┐     
q_3: ─┤ RX(2.5507) ├┤ RY(2.0146) ├──────────┤ X ├──■──
      ├────────────┤├────────────┤          └───┘┌─┴─┐
q_4: ─┤ RX(3.6771) ├┤ RY(3.3361) ├───────────────┤ X ├
      └────────────┘└────────────┘               └───┘

In [12]:
#Second unitary
size = 5
qc2 = QuantumCircuit(size)
for i in range(size):
  qc2.rx(unitary_param_2[i], i)

for i in range(size):
  qc2.ry(unitary_param_2[i+size], i)

for i in range(size-1):
  qc2.cx(i, i+1)

qc2.draw()

┌────────────┐┌────────────┐                    
q_0: ─┤ RX(3.8094) ├┤ RY(4.7641) ├──■─────────────────
      ├────────────┤├────────────┤┌─┴─┐               
q_1: ─┤ RX(5.7024) ├┤ RY(4.3118) ├┤ X ├──■────────────
     ┌┴────────────┤├────────────┤└───┘┌─┴─┐          
q_2: ┤ RX(0.42947) ├┤ RY(3.8103) ├─────┤ X ├──■───────
     └┬────────────┤├────────────┤     └───┘┌─┴─┐     
q_3: ─┤ RX(3.8595) ├┤ RY(1.9172) ├──────────┤ X ├──■──
      ├────────────┤├────────────┤          └───┘┌─┴─┐
q_4: ─┤ RX(4.9501) ├┤ RY(4.6353) ├───────────────┤ X ├
      └────────────┘└────────────┘               └───┘

In [13]:
def costf(param,numparam,numlayer):  
    circ = QuantumCircuit(12, 2)


        
    #First unitary
    circ.append(qc1, list(range(0, 5)))

    # Second unitary
    circ.append(qc2, list(range(6, 11)))

    circ.barrier()   
      # Applying the parametrized unitary

    prover = list(range(2, 6))
    for j in range(numlayer):
      for i in range(len(prover)):
        circ.rx(param[j*2*len(prover) + i], prover[i])
        circ.ry(param[j*2*len(prover) + i + len(prover)], prover[i])

      for i in range(len(prover)-1):
        circ.cx(prover[i], prover[i+1])

      circ.barrier()

    prover = list(range(8, 12))
    for j in range(numlayer):
      for i in range(len(prover)):
        circ.rx(param[j*2*len(prover) + i], prover[i])
        circ.ry(param[j*2*len(prover) + i + len(prover)], prover[i])

      for i in range(len(prover)-1):
        circ.cx(prover[i], prover[i+1])

      circ.barrier()


    circ.draw()

    backend = BasicAer.get_backend('statevector_simulator')

        # Create a Quantum Program for execution
    job = execute(circ, backend)

    result = job.result()

    outputstate = result.get_statevector(circ, decimals=11)
    
        #We have generated the output state, and we measure the first two qubits in 0.
    ket=outputstate   

    res = [ele for ele in product(range(0, 2), repeat=12)]

    for x in range(0, len(res)):
        res[x]=str(res[x][::-1]).replace(" ", "")[1::2]
        
    dic={}
    for i in range(0, len(ket)):
        dic.update({res[i]:np.real(ket[i]*np.conj(ket[i]))})
    
    res2 = ['0', '1']

    dic2={}
    for ele in res2:
        dic2[ele] = 0

    dic3={}
    for ele in res2:
        dic3[ele] = 0
        
    for ele in dic:
        string= ele[5]
        dic2[string]=dic2[string]+dic[ele]

    for ele in dic:
        string= ele[11]
        dic3[string]=dic3[string]+dic[ele]
    #cost function
    olap = 1-(1/2*(dic2['0']+dic3['1']))

    return olap


def costfp(param, numparam, index1,numlayer):
    opt_param_init = np.array([])
    for i in range(0,numparam):
        if i == index1:
            opt_param_init = np.append(opt_param_init, param[i]+np.pi/2)
        else:
            opt_param_init = np.append(opt_param_init, param[i])    
    
    return(costf(opt_param_init,numparam,numlayer))


def costfn(param, numparam, index1,numlayer):
    opt_param_init = np.array([])
    for i in range(0,numparam):
        if i == index1:
            opt_param_init = np.append(opt_param_init, param[i]-np.pi/2)
        else:
            opt_param_init = np.append(opt_param_init, param[i])    
    
    return(costf(opt_param_init,numparam,numlayer))

In [14]:
def gradient(fun1,fun2,fun3,param,numparam,numlayer,eta,eps_grad,eps_cost,count_num,gradcount):
    
    gcount=0
    count=0
    param1 = np.array([])
    costvalue = np.array([])
    dicc={}
    cost = fun1(param,numparam,numlayer)
    costvalue=np.append(costvalue, cost)
    
    num_parameters = numparam #change this if the ansatz changes
    
    for i in range(0,num_parameters):
        param1 = np.append(param1, param[i])
    while count<count_num and gcount<gradcount and cost > eps_cost:
        param2 = np.array([])
        param3 = np.array([])
        grad_vec=np.array([])
        
        for i in range(0, num_parameters):
            grad_vec =np.append(grad_vec,(fun2(param1,num_parameters,i,numlayer)-fun3(param1,num_parameters,i,numlayer))/2)
        grad=np.linalg.norm(grad_vec)**2

        if grad<=eps_grad:
            gcount+=1

        param2 = param1-eta*grad_vec

        for i in range(0,num_parameters):
            param3 = np.append(param3, param2[i])

        param3 = param2 - eta*grad_vec

        if cost - fun1(param3,num_parameters,numlayer)>=eta*grad:
            eta=2*eta
            param1 = param3
        elif cost - fun1(param3,num_parameters,numlayer)<(eta/2)*grad:
            eta = eta/2
            param1=param2
        else:
            param1=param2

        cost = fun1(param1,num_parameters,numlayer)
        alpha_opt=param1
        

        dicc.update({count:np.array([cost, alpha_opt])})
        costvalue=np.append(costvalue, cost)
        print(count, cost)
        count+=1
        
        
    return(dicc)

In [15]:
#for i in range(0,50):
opt_param_init = np.array([])

numlayer= 10
numparam = 10*numlayer

for i in range(0,numparam):
    opt_param_init = np.append(opt_param_init, np.array([rand_numb(0,2*np.pi)]))

fun1 = costf
fun2 = costfp
fun3 = costfn

ans_test = gradient(fun1,fun2,fun3,opt_param_init,numparam,numlayer,0.5,1e-8,1e-8,300,50)  

<ipython-input-14-384e8d717b31>:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dicc.update({count:np.array([cost, alpha_opt])})


0 0.36062617663540797
1 0.22694574103540655
2 0.18229497608413014
3 0.16208232735282524
4 0.15270258775469103
5 0.1416822138485052
6 0.13443591998901094
7 0.13119165688522805
8 0.12689228664777386
9 0.12498722028229925
10 0.12270879589326278
11 0.12150266666470899
12 0.11997276704124293
13 0.11912488895155815
14 0.11791953641945718
15 0.11708075613783664
16 0.11627549707034768
17 0.11574818321204483
18 0.11490813428412139
19 0.11424360980317205
20 0.11374724298247929
21 0.11337748013798743
22 0.11274555324278168
23 0.11221135191397824
24 0.11187505337864367
25 0.11139472933234096
26 0.11110862646767339
27 0.11067987887638941
28 0.11036387150993143
29 0.1100521968780912
30 0.10984084921631465
31 0.10949932317169753
32 0.1092279954063371
33 0.10901238642461752
34 0.10885265310558978
35 0.1085828074762244
36 0.10835965496419697
37 0.10824686469820355
38 0.10800361064368458
39 0.10786949552360392
40 0.10768957343951979
41 0.10758232301589521
42 0.10741210292770664
43 0.1073106042367401
44 

In [16]:
iteration = list(ans_test.keys())
cost = []  
for i,j in ans_test.values():
  cost.append(i)


In [20]:
tracedistance = 2*(2*(1-np.array(cost))-1)

In [21]:
tracedistance

array([0.55749529, 1.09221704, 1.2708201 , 1.35167069, 1.38918965,
       1.43327114, 1.46225632, 1.47523337, 1.49243085, 1.50005112,
       1.50916482, 1.51398933, 1.52010893, 1.52350044, 1.52832185,
       1.53167698, 1.53489801, 1.53700727, 1.54036746, 1.54302556,
       1.54501103, 1.54649008, 1.54901779, 1.55115459, 1.55249979,
       1.55442108, 1.55556549, 1.55728048, 1.55854451, 1.55979121,
       1.5606366 , 1.56200271, 1.56308802, 1.56395045, 1.56458939,
       1.56566877, 1.56656138, 1.56701254, 1.56798556, 1.56852202,
       1.56924171, 1.56967071, 1.57035159, 1.57075758, 1.57133732,
       1.5717179 , 1.57221627, 1.57253229, 1.57300497, 1.57333686,
       1.57371673, 1.57397009, 1.5743635 , 1.57465467, 1.57495501,
       1.57516422, 1.57549957, 1.5757588 , 1.57600394, 1.57618138,
       1.57647373, 1.57670814, 1.57691349, 1.57706732, 1.57732688,
       1.57754144, 1.57771691, 1.57785245, 1.57808594, 1.57828386,
       1.57836928, 1.57862638, 1.57876843, 1.57895178, 1.57904

In [ ]:
from qiskit.quantum_info import DensityMatrix, state_fidelity, partial_trace
psi_1 = DensityMatrix.from_instruction(qc1)
psi_2 = DensityMatrix.from_instruction(qc2)
rho_1 = partial_trace(psi_1, [0, 1])
rho_2 = partial_trace(psi_2, [0, 1])

In [ ]:
rho_1-rho_2

DensityMatrix([[ 0.11889078+6.13251854e-19j, -0.13093257-9.17610734e-02j,
                 0.07588334-5.27785364e-02j,  0.0178932 -8.96046548e-02j,
                 0.06099151+7.59633993e-02j, -0.05302161+7.80571990e-02j,
                -0.06400176+1.91481302e-01j,  0.14139228+1.24166203e-01j],
               [-0.13093257+9.17610734e-02j,  0.07557499-4.60785923e-18j,
                -0.02589225-6.35885426e-02j, -0.15949165-2.63602522e-02j,
                 0.06198644+6.49453912e-02j,  0.03717829+4.48399897e-02j,
                 0.0773468 -5.87870912e-02j, -0.10781687-2.26768787e-01j],
               [ 0.07588334+5.27785364e-02j, -0.02589225+6.35885426e-02j,
                -0.08291698-6.74915852e-18j, -0.04856702+3.99370353e-02j,
                 0.0765929 -9.23715195e-02j,  0.05558404+4.50028316e-03j,
                 0.03325011-5.39236718e-02j, -0.00667426-6.14561816e-02j],
               [ 0.0178932 +8.96046548e-02j, -0.15949165+2.63602522e-02j,
                -0.04856702-3.99370

In [ ]:
np.array(rho_1-rho_2)

array([[ 0.11889078+6.13251854e-19j, -0.13093257-9.17610734e-02j,
         0.07588334-5.27785364e-02j,  0.0178932 -8.96046548e-02j,
         0.06099151+7.59633993e-02j, -0.05302161+7.80571990e-02j,
        -0.06400176+1.91481302e-01j,  0.14139228+1.24166203e-01j],
       [-0.13093257+9.17610734e-02j,  0.07557499-4.60785923e-18j,
        -0.02589225-6.35885426e-02j, -0.15949165-2.63602522e-02j,
         0.06198644+6.49453912e-02j,  0.03717829+4.48399897e-02j,
         0.0773468 -5.87870912e-02j, -0.10781687-2.26768787e-01j],
       [ 0.07588334+5.27785364e-02j, -0.02589225+6.35885426e-02j,
        -0.08291698-6.74915852e-18j, -0.04856702+3.99370353e-02j,
         0.0765929 -9.23715195e-02j,  0.05558404+4.50028316e-03j,
         0.03325011-5.39236718e-02j, -0.00667426-6.14561816e-02j],
       [ 0.0178932 +8.96046548e-02j, -0.15949165+2.63602522e-02j,
        -0.04856702-3.99370353e-02j, -0.13135345-1.47451495e-17j,
         0.0947037 -9.70891192e-03j,  0.07444495+9.68123413e-02j,
       

In [ ]:
vw = np.linalg.eig(np.array(rho_1-rho_2))

In [ ]:
v = vw[0]

In [ ]:
v

array([-6.52571642e-01-5.31733861e-18j,  6.12231706e-01-3.32027014e-17j,
        1.80598944e-01-1.03018190e-17j, -1.40259007e-01-1.19259343e-18j,
       -3.48836970e-17+3.92652236e-17j, -3.57808176e-17-2.49683551e-17j,
        4.03115099e-17-1.65556548e-17j,  3.26971574e-17-5.22533992e-18j])

In [ ]:
tracedistance = 0

for i in v:
  tracedistance += np.abs(i)

In [ ]:
tracedistance

1.5856612982928167